In [1]:
from ehc_sn.model import EIModel, EIParameters
import matplotlib.pyplot as plt
import torch

In [2]:
from ipywidgets import interact, IntSlider, FloatSlider
from functools import partial

IntSlider = partial(IntSlider, continuous_update=False)
FloatSlider = partial(FloatSlider, continuous_update=False)

In [3]:
def plot_raster(spikes, title):
    plt.figure(figsize=(10, 5))
    for neuron_idx in range(spikes.shape[1]):
        spike_times = torch.nonzero(spikes[:, neuron_idx]).squeeze()
        plt.scatter(spike_times, neuron_idx * torch.ones_like(spike_times), s=1)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Neuron Index")
    plt.xlim(0, spikes.shape[0])
    plt.ylim(0, spikes.shape[1])
    plt.show()

In [4]:
model = EIModel()
model

EIModel(
  (excitatory): LIFRecurrentCell(input_size=100, hidden_size=100, p=LIFParameters(tau_syn_inv=tensor(200.), tau_mem_inv=tensor(100.), v_leak=tensor(0.), v_th=tensor(1.), v_reset=tensor(0.), method='super', alpha=tensor(100.)), autapses=False, dt=0.001)
  (inihibitory): LIFCell(p=LIFParameters(tau_syn_inv=tensor(200.), tau_mem_inv=tensor(100.), v_leak=tensor(0.), v_th=tensor(1.), v_reset=tensor(0.), method='super', alpha=tensor(100.)), dt=0.001)
)

In [5]:
@interact(
    current=FloatSlider(min=0, max=10, step=0.1, value=2.4),
    t1=IntSlider(min=0, max=1000, step=1, value=200),
    t2=IntSlider(min=0, max=1000, step=1, value=300),
)
def experiment(current, t1, t2):
    model.reset()
    input_currents = torch.zeros(1000, model.excitatory.input_size)
    input_currents[t1:t2, 0] = current  # Apply current to the first neuron
    outputs = model(input_currents)
    plot_raster(outputs, "Balanced state spikes")

interactive(children=(FloatSlider(value=2.4, continuous_update=False, description='current', max=10.0), IntSli…

In [6]:
@interact(
    rr_conn=FloatSlider(min=0, max=0.2, step=0.001, value=0.1),
    ei_prop=FloatSlider(min=0, max=0.4, step=0.001, value=0.1),
    ie_prop=FloatSlider(min=0, max=0.4, step=0.001, value=0.1),
)
def experiment(rr_conn, ei_prop, ie_prop):
    model = EIModel(EIParameters(rr_conn=rr_conn, ei_prop=ei_prop, ie_prop=ie_prop))
    input_currents = torch.zeros(1000, model.excitatory.input_size)
    input_currents[100:300, 0] = 2.4  # Apply current to the first neuron
    outputs = model(input_currents)
    plot_raster(outputs, "Balanced state spikes")

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='rr_conn', max=0.2, step=0.0…